In [1]:
%matplotlib inline
import pandas as pd
import json
import numpy as np
from datetime import datetime, date, timedelta

# Calibration

This notebook can be used to set the baseline parameters of the model. It generates the base parameter.json file that is used for all simulations.

## 1 Set the general simulation parameters 

In [2]:
i = 0
TIME = 300 #350
AGENTS = 25000#00 
MONTE_CARLO_RUNS = 1
CITY = ['cape_town', 'johannesburg'][i]
REGION = ['Western Cape', 'Gauteng'][i]

## 2 Set the start and end dates for the validation period

In [3]:
city_dates = ['2020-04-17', '2020-04-29'] 
START_DATE = datetime.strptime(city_dates[i], '%Y-%m-%d')
END_DATE = datetime.strptime('2020-08-10', '%Y-%m-%d') 

In [4]:
END_DATE + timedelta(days=36)

datetime.datetime(2020, 9, 15, 0, 0)

## 3 Import data

### 3.1 Oxford Stringency Index

In [5]:
stringency_index = pd.read_csv('OxCGRT_latest.csv')[pd.read_csv('OxCGRT_latest.csv')['CountryCode'] == 'ZAF']
stringency_index.index = [datetime.strptime(str(x), '%Y%m%d') for x in stringency_index['Date']]
stringency_index = stringency_index['StringencyIndex']

In [6]:
lockdown_level_5 = datetime(2020, 3, 26) # lockdown level 5
lockdown_level_4 = datetime(2020, 5, 1) # lockdown level 4
lockdown_level_3 = datetime(2020, 6, 1) # lockdown level 3

In [7]:
lockdown_index = []
for idx, x in enumerate(stringency_index.index):
    if x < lockdown_level_5: 
        #print(x, 0)
        lockdown_index.append(0.0)
    elif x >= lockdown_level_5 and x < lockdown_level_4:
        #print(x, 80.0)
        lockdown_index.append(30.0)
    elif x >= lockdown_level_4 and x < lockdown_level_3:
        #print(x, 60.0)
        lockdown_index.append(22.0)
    else: 
        #print(x, 40.0)
        lockdown_index.append(18.0)
lockdown_index = pd.Series(lockdown_index)
lockdown_index.index = stringency_index.index

In [8]:
lockdown_severeness = lockdown_index.loc[START_DATE:END_DATE]

### 3.2 Google mobility data

In [9]:
mobility_data = pd.read_csv('Global_Mobility_Report_ZA.csv')
mobility_data = mobility_data[mobility_data['sub_region_1'] == REGION]
mobility_data.index = [datetime.strptime(x, '%Y-%m-%d') for x in mobility_data['date']]
mobility_data = mobility_data[mobility_data.columns[9:]].astype(float)

## 3 Set policy parameters for the duration of the simulation.
Policy parameters are input as a list that is as long as the simulation. This way they can change over the course of the simulation, in line with observed policy. 

The travel multiplier is set using the Google mobility data.

In [10]:
travel_multipliers = list(1 + mobility_data.mean(axis=1).loc[START_DATE:END_DATE] / 100)
#travel_multipliers

The maximum number of contacts allowed by the government is based on three trips in a mini bus taxi. A taxi holds 16 people in total (so 15 besides the driver). This is multiplied by 70% which the South African government stipulated was allowed. 

In [11]:
l_awareness = 0.95
l_awareness

0.95

In [12]:
gathering_max_contacts = int(round((15 * 0.7 + 1)))# * 3
gathering_max_contacts

12

In [13]:
p_dist_mult = 0.8
p_dist_mult

0.8

## 4 Set initial infections and age groups

Next, we assume that 3% of infections were detected at the start of the simulation and translate this to the initial number of cases at the start of the simulation. 

In [14]:
perc_infections_detects = 3
initial_agents = max(round((310 / (3740026 / AGENTS) * 100 / perc_infections_detects)), 20) # 310 cases / (population / agent) * 1 / 14% detected cases

initial_agents = 100 # comment this out later
initial_agents

100

The age groups are per decile. 

In [15]:
age_groups = ['age_0_10', 'age_10_20', 'age_20_30', 'age_30_40', 'age_40_50', 
              'age_50_60', 'age_60_70', 'age_70_80', 'age_80_plus']

Health system capacity city 

In [16]:
beds_joburg = 8750 / 12500000
beds_cape_town = 0.0009179
health_system_capacities = [beds_cape_town, beds_joburg]
health_system_capacities

[0.0009179, 0.0007]

## 5 Create the parameters 

In [17]:
parameters = {
    # Parameters related to model implementation
    "time": TIME, 
    "number_of_agents": AGENTS,
    "monte_carlo_runs": MONTE_CARLO_RUNS,
    
    # COVID-19 parameters (9)
    "exposed_days": 4, # (not changed) average number of days before being able to infect others (sources: NICD + CDC)
    "asymptom_days": 7, # (used to be 10) average number of days agents are infected but do not have symptoms 
    "symptom_days": 7,# (used to be 10) average number of days agents with mild symptoms are infectious (NICD = 7, Balabdaoui and Mohr = 8, Huang et al=7)
    "critical_days": 11, # (used to be 8) average number of days agents are in critical condition (Balabdaoui and Mohr = 8, NICD=8-19 (13.5), CDC=10-14 (12))
    "probability_symptomatic": (1 - 0.6165), # (not changed) determines whether an agent will become asymptomatic or asymptomatic spreader
    "no_hospital_multiplier": 1.79, # the increase in probability if a critical agent cannot go to the hospital SOURCE: Zhou et al. 2020
    "probability_transmission": 0.03, #0.005,#0.00335, # should be estimated to replicate realistic R0 number.
    "probability_critical": {key:value for key, value in zip(age_groups, [0.001, 0.003, 0.012, 0.032, 0.049, 0.102, 0.166, 0.244, 0.273])}, # probability that an agent enters a critical stage of the disease SOURCE: Verity et al.
    "probability_to_die": {key:value for key, value in zip(age_groups, [0.02090209, 0.032569361, 0.034233668, 0.052638239, 0.097470817, 0.155112718, 0.248512233, 0.306164902, 0.371187541])}, #used to be [0.005, 0.021, 0.053, 0.126, 0.221, 0.303, 0.565, 0.653, 0.765])}, probability to die per age group in critical stage SOURCE: Verity et al.
    
    # learning parameters
    'private_shock_stdev': 0.05,
    'weight_private_signal': 0.15,
    
    # Cape Town specific parameters (2)
    "health_system_capacity": health_system_capacities[i],
    "stringency_index": list(lockdown_severeness),
    # Policy parameters
    # (1) physical distancing measures such as increased hygiëne & face mask adoption 
    "physical_distancing_multiplier": p_dist_mult,#inf_multiplier, # 0.31 was based on a study of face mask on hamsters by Yuen et al. (2020)
    # (2) reducing travel e.g. by reducing it for work, school or all
    "visiting_recurring_contacts_multiplier": travel_multipliers,#[travel_multiplier for x in range(0, TIME)], # based on travel data
    # (3) Testing and general awareness
    'likelihood_awareness': l_awareness,#likelihood_awareness, #li2020early this will be increased through testing, track & trace and coviid
    # (4) limiting mass contact e.g. forbidding large events, outside household. 
    "gathering_max_contacts": gathering_max_contacts, #max_contacts, # based on the regulations for mini bus taxis --> (15 * 0.7) + driver
    
    # initial infections
    "total_initial_infections": initial_agents, # total agents infected in CT
    
    # optional parameters for second wave
    'time_4_new_infections': -1, # -1 is never
    'new_infections_scenario': 'None',
    
    # additional parameter used to switch of informal districts
    "informality_dummy": 1.0, # setting this parameter at 0 will mean the lockdown is equally effective anywhere, alternative = 1
    
    # Technical parameters
    'init_infected_agent': 0, # to calculate R0
    "data_output": 'csv-light', # 'csv', 'csv-light' or 'network', or 'False'
    
    # Depreciated paramters (can be used later)
    "probability_susceptible": 0.000, # probability that the agent will again be susceptible after having recovered
}

Next, we store these parameters in a .json file.

In [18]:
parameters

{'time': 300,
 'number_of_agents': 25000,
 'monte_carlo_runs': 1,
 'exposed_days': 4,
 'asymptom_days': 7,
 'symptom_days': 7,
 'critical_days': 11,
 'probability_symptomatic': 0.38349999999999995,
 'no_hospital_multiplier': 1.79,
 'probability_transmission': 0.03,
 'probability_critical': {'age_0_10': 0.001,
  'age_10_20': 0.003,
  'age_20_30': 0.012,
  'age_30_40': 0.032,
  'age_40_50': 0.049,
  'age_50_60': 0.102,
  'age_60_70': 0.166,
  'age_70_80': 0.244,
  'age_80_plus': 0.273},
 'probability_to_die': {'age_0_10': 0.02090209,
  'age_10_20': 0.032569361,
  'age_20_30': 0.034233668,
  'age_30_40': 0.052638239,
  'age_40_50': 0.097470817,
  'age_50_60': 0.155112718,
  'age_60_70': 0.248512233,
  'age_70_80': 0.306164902,
  'age_80_plus': 0.371187541},
 'private_shock_stdev': 0.05,
 'weight_private_signal': 0.15,
 'health_system_capacity': 0.0009179,
 'stringency_index': [30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  2

In [19]:
with open('{}/parameters.json'.format(CITY), 'w') as outfile:
    json.dump(parameters, outfile)

In [20]:
with open('config_ct.json', 'w') as outfile:
    json.dump(parameters, outfile)